### Scarichiamo le principali librerie

In [2]:
%pip install selenium
%pip install webdriver-manager
%pip install requests beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\burre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\burre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\burre\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


### Importiamo le principali librerie da usare

In [6]:
import os
import time
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import locale
import datetime
from selenium.webdriver.firefox.service import Service
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from lxml import etree
import re

### Settiamo le impostazioni di Selenium per la ricerca su browser

In [ ]:
# Firefox browser settings
options = Options()
options.add_argument("--headless")  # Force headless mode (no GUI)
service = Service()
# Headers to simulate a request from a real browser (Firefox)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:89.0) Gecko/20100101 Firefox/89.0"
}
driver = webdriver.Firefox(service=service, options=options)

### Definiamo le principali funzioni per estrapolare i link e le informazioni dalle note degli aggiornamenti del decreto / legge

In [ ]:
# Function to open and verify the link
def open_and_check_link(link):
    driver.get(link)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    # Search for the updates button with id="aggiornamenti_atto_button"
    update_button = soup.find('a', id='aggiornamenti_atto_button')
    if update_button:
        data_href = update_button.get('data-href')
        updates_url = urljoin("https://www.normattiva.it/", data_href)
        print(f"Updates link found: {updates_url}")
        driver.get(updates_url)
        time.sleep(2)

        soup_updates = BeautifulSoup(driver.page_source, 'html.parser')
        updates_table = soup_updates.find('table')
        updates_data = []

        if updates_table:
            # Temporary variables to maintain the most recent date and name of the decree
            current_date = None
            current_decree_name = None
            
            for row in updates_table.find_all('tr'):
                date_td = row.find('td', class_='sinistra')
                if date_td and date_td.get_text(strip=True):
                    current_date = date_td.get_text(strip=True)
            
                date = current_date
            
                decree_td = row.find('td', class_='destra')
                if decree_td:
                    ul = decree_td.find('ul')
                    if ul:
                        li_elements = ul.find_all('li')

                        # If is the first line of <tr>
                        if len(li_elements) == 2:
                            current_decree_name = li_elements[0].get_text(strip=True)
                            modification_a = li_elements[1].find('a')
                        elif len(li_elements) == 1:
                            modification_a = li_elements[0].find('a')

                        modification_link = modification_a['href'] if modification_a else None
                        modification_name = modification_a.get_text(strip=True) if modification_a else None
            
                        updates_data.append({
                            'data': date,
                            'nome_decreto': current_decree_name,
                            'nome_modifica': modification_name,
                            'link_modifica': modification_link
                        })
    
            # Group the data by date and decree name
            grouped_updates = {}
            for update in updates_data:
                date = update['data']
                decreto = update['nome_decreto']
                key = (date, decreto)
                if key not in grouped_updates:
                    grouped_updates[key] = []
                grouped_updates[key].append(update)
        
            return grouped_updates
        else:
            print("No updates table found.")
            return None
    else:
        print("No updates button found on the page.")
        return None

def clean_folder_name(name):
    name = name.replace("/", "-")
    name = re.sub(r'[^a-zA-Z0-9()\s-]', ' ', name)
    name = re.sub(r'\s+', ' ', name).strip()
    return name

def extract_info(nome_modifica):
    match = re.search(r"disposto\s*\(.*?l'art\.\s*(\d+)(?:,\s*comma\s*(\d+))?(?:,\s*lettera\s*([a-z]+))?", nome_modifica)
    if match:
        articolo = match.group(1)
        comma = match.group(2) if match.group(2) else None
        lettera = match.group(3) if match.group(3) else None
        return articolo, comma, lettera
    return None, None, None

def get_content_with_links(element):
    if not element:
        return ""
    
    content_parts = []
    
    if element.string:
        content_parts.append(element.string.strip().replace('\n', ' '))

    for child in element.children:
        if child.name == 'a':
            content_parts.append(child.get_text(strip=True).replace('\n', ' '))
            if child.next_sibling:
                content_parts.append(child.next_sibling.strip().replace('\n', ' '))
        else:
            if isinstance(child, str):
                content_parts.append(child.strip().replace('\n', ' '))

    return ' '.join(content_parts)

# Function for convert a letter into a number
def letter_to_number(letter):
    alphabet = "abcdefghilmnopqrstuvz"
    letter_to_num = {char: i + 1 for i, char in enumerate(alphabet)}

    # Gestione di singola lettera (a, b, c, etc.)
    if letter in letter_to_num:
        return letter_to_num[letter]

    # Gestione di lettere doppie (aa, bb, cc, etc.)
    if len(letter) == 2 and letter[0] == letter[1] and letter[0] in alphabet:
        base_value = letter_to_num[letter[0]]
        return 21 + (base_value - 1)

    # Se non viene trovata alcuna corrispondenza, restituisce None
    return None

def save_updates(updates_data, nome_cartella):
    base_folder = os.path.expanduser('~/Desktop/AggiornamentiNormattiva')
    specific_folder = os.path.join(base_folder, clean_folder_name(nome_cartella))

    if os.path.exists(specific_folder):
        for filename in os.listdir(specific_folder):
            file_path = os.path.join(specific_folder, filename)
            os.remove(file_path)
    else:
        os.makedirs(specific_folder, exist_ok=True)

    for (date, decreto), items in updates_data.items():
        group_folder_name = f"{date} {decreto}"
        group_folder_path = os.path.join(specific_folder, clean_folder_name(group_folder_name))
        os.makedirs(group_folder_path, exist_ok=True)

        for index, item in enumerate(items):
            nome_modifica = clean_folder_name(item['nome_modifica'])
            link_modifica = "https://www.normattiva.it" + item['link_modifica']
        
            modification_type = ""
            if 'corrige' in decreto.lower():
                modification_type = "corrige"
            elif 'rettifica' in decreto.lower():
                modification_type = "rettifica"
            elif 'corte' in decreto.lower():
                modification_type = "corte"
            elif 'decreto' in decreto.lower():
                modification_type = "decreto"
            elif 'legge' in decreto.lower():
                modification_type = "legge"

            response = requests.get(link_modifica)
            soup = BeautifulSoup(response.text, 'html.parser')

            content = ""

            if modification_type in ["corrige", "rettifica", "corte"]:
                pre_tags = soup.find_all('pre')
                content = "\n".join(pre.get_text(strip=True).replace('\n', ' ') for pre in pre_tags if pre)
            elif modification_type in ["decreto", "legge"]:
                articolo, comma, lettera = extract_info(item['nome_modifica'])

                if articolo:
                    contents = soup.find_all(class_="art-comma-div-akn")
                    if not contents:
                        content_div = soup.find(class_="art-just-text-akn")
                        if content_div:
                            content = get_content_with_links(content_div)
                    else:
                        if comma:
                            content = get_content_with_links(contents[int(comma) - 1]) if len(contents) >= int(comma) else ""
                        else:
                            content = get_content_with_links(contents[0])

                        if lettera:
                            pointed_list_first = contents[0].find(class_="pointedList-first-akn")
                            pointed_list_rest = contents[0].find_all(class_="pointedList-rest-akn")
                            
                            if pointed_list_first:
                                content += "\n" + get_content_with_links(pointed_list_first)

                            index = letter_to_number(lettera) -1
                            
                            if index is not None and index < len(pointed_list_rest):
                                item = pointed_list_rest[index] 
                                content += "\n" + get_content_with_links(item)
                        else:
                            art_text = contents[0].find(class_="art_text_in_comma")
                            content += "\n" + (get_content_with_links(art_text) if art_text else "")
                else:
                    content = "No article found."

            else:
                content = "Content not specified for this type of modification."

            with open(os.path.join(group_folder_path, f"{index}.txt"), 'w', encoding='utf-8') as f:
                f.write(f"{nome_modifica}\n{content}\n")

            print(f"{nome_modifica} downloaded")

### Eseguiamo lo script per scaricare tutti gli aggiornamenti dei decreti e delle leggi scaricate precedentemente (i riferimenti sono salvati in un file .txt)

In [7]:
updates_folder = os.path.join(os.path.expanduser('~'), 'Desktop', 'AggiornamentiNormattiva')

if not os.path.exists(updates_folder):
    os.makedirs(updates_folder)
    print(f"'AggiornamentiNormattiva' folder created: {updates_folder}")
else:
    print(f"'AggiornamentiNormattiva' folder already exists: {updates_folder}")

normattiva_links_file = os.path.join(os.path.expanduser('~'), 'Desktop', 'NormattivaLinks.txt')

link_folder_list = []

if os.path.exists(normattiva_links_file):
    with open(normattiva_links_file, 'r') as f:
        lines = f.readlines()
    
    for line in lines:
        line = line.strip()
        
        parts = line.split(';')
        
        if len(parts) >= 3:
            link = f"{parts[0]};{parts[1]}"
            folder_name = ";".join(parts[2:])

            link_folder_list.append({'link': link, 'nome_cartella': folder_name})
            print(f"Link: {link}, Folder name: {folder_name}")

    for item in link_folder_list:
        print(f"Opening the link: {item['link']}")
        data = open_and_check_link(item['link'])
        save_updates(data, item['nome_cartella'])
else:
    print("The file NormattivaLinks.txt is not present on the desktop.")

driver.quit()


'AggiornamentiNormattiva' folder created: C:\Users\burre\Desktop\AggiornamentiNormattiva
Link: https://www.normattiva.it/uri-res/N2Ls?urn:nir:stato:decreto.del.presidente.della.repubblica:1988-09-22;447, Folder name: DECRETO DEL PRESIDENTE DELLA REPUBBLICA 22 settembre 1988,n. 447
Opening the link: https://www.normattiva.it/uri-res/N2Ls?urn:nir:stato:decreto.del.presidente.della.repubblica:1988-09-22;447
Updates link found: https://www.normattiva.it/do/atto/vediAggiornamentiAllAtto?atto.dataPubblicazioneGazzetta=1988-10-24&atto.codiceRedazionale=088G0492
ha disposto (con l art 23 comma 2) la modifica dell art 303 downloaded
relativo all art 57 comma 3 downloaded
relativo all art 301 comma 1 downloaded
relativo all art 534 comma 1 downloaded
relativo all art 722 comma 2 downloaded
ha disposto (con l art 1 comma 1) la modifica dell art 125 comma 5 downloaded
ha dichiarato l illegittimit costituzionale dell art 452 comma 2 downloaded
ha disposto (con l art 19 comma 1) l introduzione del c

KeyboardInterrupt: 